In [2]:
#!sudo pip install --upgrade transformers

In [2]:
from refractml import *
from refractml.constants import MLModelFlavours
# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
payload = {"payload":[{"date_info":{
           "start_date": "01/01/2020",
          "end_date": "15/09/2023"
},"question":"top 2 segments and bottom 3 sub-category basis quantity","context":{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}}]}

In [4]:
# import json
# print(json.dumps(payload,indent=2))

In [5]:
context ={
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [6]:
context ={
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
                         "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
                        {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
                        {"ENTITY": "contribution_to_growth",
                         "other names": ["contribution to growth", "growth", "grown"]},
                        {"ENTITY": "kda_transactional",
                         "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
                        {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
                        {"ENTITY": "correlation",
                         "other names": ["associate", "associated", "association", "associations", "correlate",
                                         "correlated",
                                         "correlation", "correlations", "relate", "related", "relation", "relations",
                                         "relationship",
                                         "relationships"]}
                        ],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [ ]:
 """You are an assistant that helps to map the user question to the context for a question answering system.
         You might also need to act as a time tagger expert to convert the date elements present in the question to a standard format and to find possible date ranges for the same.
        Step 1: Identify the n-grams match between question and context
        Map the n-gram or their lemma or their inflections from the question with the 'other names' in the passed context.
        Always consider the longest n-gram match, not the sub-string.
        If there are multiple matches for an n-gram with context, return all such ENTITY in response.
        If you are returning any match which is not exactly present with the 'other names', make sure that it is a noun phrase and there is a high similarity between the match and the matched "ENTITY". 
        Step 2: Applying other conditions
        Once the match is identified, next step is to identify other conditions from user question and apply it to the identified matches.
        Refer to the following statements to understand about different types of conditions to be applied:
            1. METRIC CONSTRAINT : METRIC can be MEASURE or DERIVED MEASURE. User is asking for a comparison limit to be applied on the METRIC. It has two parts: "COMPARISON VALUE" is the value applied on a METRIC and "COMPARSION OPERATOR" is the operator (in symbols) applied between METRIC and COMPARISON VALUE.
            2. ADJECTIVE and TONE : Identify the adjectives (like least, highest performing etc.) applied on the matched ENTITY. TONE is the intent of adjective and it can be postive or negative.
            3. EXCEPTION : Excluded FILTER of a DIMENSION asked in question if any. DIMENSION should be the parent of the FILTER. Add a key "EXCLUDE" for such excluded FILTERS and set the value as "True" in the response.
            4. RANK : Rank applied on a DIMENSION if any like top 5, bottom 3 etc. It has two parts: "RANK ADJECTIVE", the adjective like top, bottom etc. and "RANK VALUE", a number that comes along with the RANK ADJECTIVE, immediately before or after. If there is no explicit RANK VALUE in question, make it as 1. Based on the meaning of the RANK ADJECTIVE, make it as either top or bottom.
            5. RATIO FILTERS : This is applicable only for ENTITY "Ratio" in DERIVED MEASURE. Identify the FILTER on which Ratio needs to be calculated. Example 1: Question: bike share of sales in area, (where ENTITY of 'bike' is 'Bikes'), RATIO FILTERS = [{'bike': 'Bikes'}]. Example 2: Question: in area, share of bike and cycle basis sales, (where ENTITY of 'Bike' is 'Bikes' and 'cycle' is 'Cycle') RATIO FILTERS = [{'bike': 'Bikes', 'cycle': 'Cycles'}]. If there are no matched FILTERS, then keep RATIO FILTERS = []"
            6. APPLIED MEASURES: This is applicable only for DERIVED MEASURE. Identify the MEASURE on which the DERIVED MEASURE needs to be calculated. 

            Step 3: Applying time tagger rules only if time elements are present in question

            Identify the TIME ELEMENTS in the input question and convert it to a standard format (if not already) by applying the general time tagging rules. If the TIME ELEMENT is already in a standard format, then no need to convert it.
            TIME ELEMENT can be either a temporal interval (across months, yoy, mom, qoq, wow, quarterly etc.) or a temporal expression (time points such as specific dates, relative expressions etc.).
            Calculate date range for each time points based on the following conditions:
            1. For relative time expressions, calculate the date range based on a reference date - By default the reference date is the end_date in date input: """ '\n' +  '@#<date_input>@#' + '\n' """
            2. To calculate the date range for "last X years", strictly follow the below conditions:
                    For "last 1 year", consider exactly one year before the reference year and set start date as January 1 and end date as Decemebr 31 of that year.
                    For "last X years", where X is greater than 1, consider starting year = (reference year - X+1) and set start date as January 1 of starting year and end date as the reference date.
            3. To calculate the date range for "last X months", strictly follow the below conditions:
                    Consider the reference month as the month in reference date
                    For "last 1 month", consider exactly one month before the reference month and set start date as first day and end date as last day of that month .
                    For "last X months", where X is greater than 1, consider starting month = (reference month - X+1) and set start date as first day of starting month and end date as the reference date. (Example: if reference date is 14/09/2022, then last 3 months = 01/07/2022 - 14/09/2022)
            4. To calculate the date range for "last X quarters", strictly follow the below conditions:
                    For "last 1 quarter", consider exactly one quarter before the reference quarter and set start date as first day and end date as last day of that quarter .
                    For "last X quarter", where X is greater than 1, consider starting quarter = (reference quarter - X+1) and set start date as first day of starting quarter and end date as the reference date.
            5. To calculate the date range for "last X weeks", strictly follow the below conditions:
                    For "last 1 week", consider exactly one week before the reference week and set start date as Monday and end date as Sunday of that week .
                    For "last X weeks", where X is greater than 1, consider starting week = (reference week - X+1) and set start date as Monday of starting week and end date as the reference date.
            6. Provide the date range of each time point in start date - end date format always.

            Step 4: Creating the response JSON
            Strictly return the response in the exact same JSON format as follows. 
            Fill the information identified from above steps in the JSON. 
            The keys mentioned in upper case in the response are constant.
            Return only if match is found from the context and non empty values are present.
            Replace the placeholders "<>" with your findings.
            {
                "MEASURE": {
                    "<n-gram matched to MEASURE>": [
                        {
                            "ENTITY": "<Matched MEASURE>",
                            "MEASURE CONSTRAINT": [
                                {
                                    "COMPARISON VALUE": "",
                                    "COMPARSION OPERATOR": ""
                                }
                            ],
                            "ADJECTIVE": [],
                            "TONE": ""
                        }
                    ]
                },
                "DIMENSION": {
                    "<n-gram matched to DIMENSION>": [
                        {
                            "ENTITY": "<Matched DIMENSION>",
                            "RANK": [{"RANK ADJECTIVE":"", "RANK VALUE": ""}],
                            "ADJECTIVE": [],
                            "TONE": ""
                        }
                    ]
                },
                "FILTER": {
                    "<n-gram matched to FILTER>": [
                        {
                            "ENTITY": "<Matched FILTER>",
                            "PARENT": "<parent of the Matched FILTER>",
                            "EXCLUDE": ""
                        }
                    ]
                },
                "DERIVED MEASURE": {
                    "<n-gram matched to DERIVED MEASURE>": [
                        {
                            "ENTITY": "<Matched DERIVED MEASURE>",
                            "RATIO FILTER": [{}],
                            "APPLIED MEASURE": [{"<n-gram matched to MEASURE>": "<Matched MEASURE>"}],
                            "DERIVED MEASURE CONSTRAINT": [
                                {
                                    "COMPARISON VALUE": "",
                                    "COMPARSION OPERATOR": ""
                                }
                            ],
                            "ADJECTIVE": [],
                            "TONE": ""
                        }
                    ]
                },
                "DATE VARIABLE": {
                    "asked time element": [{"ENTITY": "<Matched DATE VARIABLE>"
                        "DATE RANGE": "date range",
                        "CONVERTED TIME ELEMENT": "<converted time element>"
                        }]
                }
            }

            Provide reasoning

In [7]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        self.model_loaded = None
        self.model_name ="/data/huggingface/cache/models--openchat--openchat_3.5/snapshots/2d39b2e68fc16446e1760700e0de14721434cbbf"
        #self.date_input = {
        #    "start_date": "01/01/2020",
        #   "end_date": "15/09/2023"}

        self.base_prompt = """You are an assistant that helps to map the user question to the context for a question answering system.
         You might also need to act as a time tagger expert to convert the date elements present in the question to a standard format and to find possible date ranges for the same.
        Step 1: Identify the n-grams match between question and context
        Map the n-gram or their lemma or their inflections from the question with the 'other names' in the passed context.
        Always consider the longest n-gram match, not the sub-string.
        If there are multiple matches for an n-gram with context, return all such ENTITY in response.
        If you are returning any match which is not exactly present with the 'other names', make sure that it is a noun phrase and there is a high similarity between the match and the matched "ENTITY". 
        Step 2: Applying other conditions
        Once the match is identified, next step is to identify other conditions from user question and apply it to the identified matches.
        Refer to the following statements to understand about different types of conditions to be applied:
            1. METRIC CONSTRAINT : METRIC can be MEASURE or DERIVED MEASURE. User is asking for a comparison limit to be applied on the METRIC. It has two parts: "COMPARISON VALUE" is the value applied on a METRIC and "COMPARSION OPERATOR" is the operator (in symbols) applied between METRIC and COMPARISON VALUE.
            2. ADJECTIVE and TONE : Identify the adjectives (like least, highest performing etc.) applied on the matched ENTITY. TONE is the intent of adjective and it can be postive or negative.
            3. EXCEPTION : Excluded FILTER of a DIMENSION asked in question if any. DIMENSION should be the parent of the FILTER. Add a key "EXCLUDE" for such excluded FILTERS and set the value as "True" in the response.
            4. RANK : Rank applied on a DIMENSION if any like top 5, bottom 3 etc. It has two parts: "RANK ADJECTIVE", the adjective like top, bottom etc. and "RANK VALUE", a number that comes along with the RANK ADJECTIVE, immediately before or after. If there is no explicit RANK VALUE in question, make it as 1. Based on the meaning of the RANK ADJECTIVE, make it as either top or bottom.
            5. RATIO FILTERS : This is applicable only for ENTITY "Ratio" in DERIVED MEASURE. Identify the FILTER on which Ratio needs to be calculated. Example 1: Question: bike share of sales in area, (where ENTITY of 'bike' is 'Bikes'), RATIO FILTERS = [{'bike': 'Bikes'}]. Example 2: Question: in area, share of bike and cycle basis sales, (where ENTITY of 'Bike' is 'Bikes' and 'cycle' is 'Cycle') RATIO FILTERS = [{'bike': 'Bikes', 'cycle': 'Cycles'}]. If there are no matched FILTERS, then keep RATIO FILTERS = []"
            6. APPLIED MEASURES: This is applicable only for DERIVED MEASURE. Identify the MEASURE on which the DERIVED MEASURE needs to be calculated. 

            Step 3: Applying time tagger rules only if time elements are present in question

            Identify the TIME ELEMENTS in the input question and convert it to a standard format (if not already) by applying the general time tagging rules. If the TIME ELEMENT is already in a standard format, then no need to convert it.
            TIME ELEMENT can be either a temporal interval (across months, yoy, mom, qoq, wow, quarterly etc.) or a temporal expression (time points such as specific dates, relative expressions etc.).
            Calculate date range for each time points based on the following conditions:
            1. For relative time expressions, calculate the date range based on a reference date - By default the reference date is the end_date in date input: """ '\n' +  '@#<date_input>@#' + '\n' """
            2. To calculate the date range for "last X years", strictly follow the below conditions:
                    For "last 1 year", consider exactly one year before the reference year and set start date as January 1 and end date as Decemebr 31 of that year.
                    For "last X years", where X is greater than 1, consider starting year = (reference year - X+1) and set start date as January 1 of starting year and end date as the reference date.
            3. To calculate the date range for "last X months", strictly follow the below conditions:
                    Consider the reference month as the month in reference date
                    For "last 1 month", consider exactly one month before the reference month and set start date as first day and end date as last day of that month .
                    For "last X months", where X is greater than 1, consider starting month = (reference month - X+1) and set start date as first day of starting month and end date as the reference date. (Example: if reference date is 14/09/2022, then last 3 months = 01/07/2022 - 14/09/2022)
            4. To calculate the date range for "last X quarters", strictly follow the below conditions:
                    For "last 1 quarter", consider exactly one quarter before the reference quarter and set start date as first day and end date as last day of that quarter .
                    For "last X quarter", where X is greater than 1, consider starting quarter = (reference quarter - X+1) and set start date as first day of starting quarter and end date as the reference date.
            5. To calculate the date range for "last X weeks", strictly follow the below conditions:
                    For "last 1 week", consider exactly one week before the reference week and set start date as Monday and end date as Sunday of that week .
                    For "last X weeks", where X is greater than 1, consider starting week = (reference week - X+1) and set start date as Monday of starting week and end date as the reference date.
            6. Provide the date range of each time point in start date - end date format always.

            Step 4: Creating the response JSON
            Strictly return the response in the exact same JSON format as follows. 
            Fill the information identified from above steps in the JSON. 
            The keys mentioned in upper case in the response are constant.
            Return only if match is found from the context and non empty values are present.
            Replace the placeholders "<>" with your findings.
            {
                "MEASURE": {
                    "<n-gram matched to MEASURE>": [
                        {
                            "ENTITY": "<Matched MEASURE>",
                            "MEASURE CONSTRAINT": [
                                {
                                    "COMPARISON VALUE": "",
                                    "COMPARSION OPERATOR": ""
                                }
                            ],
                            "ADJECTIVE": [],
                            "TONE": ""
                        }
                    ]
                },
                "DIMENSION": {
                    "<n-gram matched to DIMENSION>": [
                        {
                            "ENTITY": "<Matched DIMENSION>",
                            "RANK": [{"RANK ADJECTIVE":"", "RANK VALUE": ""}],
                            "ADJECTIVE": [],
                            "TONE": ""
                        }
                    ]
                },
                "FILTER": {
                    "<n-gram matched to FILTER>": [
                        {
                            "ENTITY": "<Matched FILTER>",
                            "PARENT": "<parent of the Matched FILTER>",
                            "EXCLUDE": ""
                        }
                    ]
                },
                "DERIVED MEASURE": {
                    "<n-gram matched to DERIVED MEASURE>": [
                        {
                            "ENTITY": "<Matched DERIVED MEASURE>",
                            "RATIO FILTER": [{}],
                            "APPLIED MEASURE": [{"<n-gram matched to MEASURE>": "<Matched MEASURE>"}],
                            "DERIVED MEASURE CONSTRAINT": [
                                {
                                    "COMPARISON VALUE": "",
                                    "COMPARSION OPERATOR": ""
                                }
                            ],
                            "ADJECTIVE": [],
                            "TONE": ""
                        }
                    ]
                },
                "DATE VARIABLE": {
                    "asked time element": [{"ENTITY": "<Matched DATE VARIABLE>"
                        "DATE RANGE": "date range",
                        "CONVERTED TIME ELEMENT": "<converted time element>"
                        }]
                }
            }

            Provide reasoning
            """
        from transformers import pipeline
        import time
        import torch
        from huggingface_hub.utils import disable_progress_bars
        disable_progress_bars()
        if self.model_loaded is None:
            print("LLM model loading Open_Chat_3_5")
            self.model_loaded = pipeline("text-generation", model= self.model_name, torch_dtype=torch.bfloat16, device_map="auto",model_kwargs={"offload_folder":"offload"})
            
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[{"question": "","context":""},{"question": "","context":""},...]

        :return: (n_inputs, payload's)

        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (len(raw_payload), raw_payload) 
    
    def pre_processing_fn(self,payload):
        return payload

    
    def prediction_fn(self,
                      model: Any,
                      input_query
                       ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        question = input_query.get("question")
        context = input_query.get("context")
        date_info = input_query.get("date_info")
        query = """Now Based on this following context: \n """+ str(context) + "\n return a response for this question:\n" + question + "\n"
        prompt = f"""<s>[IDENTITY]
                    {self.base_prompt.replace('@#<date_input>@#',str(date_info))}
                    [/IDENTITY]</s>
                    [INST]
                    {query}
                    [/INST]
                    """
        print("Prompting has started")
        import time
        start = time.time()
        response = self.model_loaded(prompt, max_new_tokens=1024*4)#.to("cuda")
        print(f"Total time taken = {time.time()-start}")
        response = response[0]["generated_text"]
        preds = response.split("[SOL]")[-1].split("[/SOL]")[0]
        print("prediction is \n: ",preds)
        return [preds]

In [8]:
score_ = ScoreTemplateExample()
req = requests.Request()
req.json = {"payload":[{"question":"top 2 segments and bottom 3 sub-category basis quantity","context":context,"date_info":{"start_date": "01/01/2020","end_date": "15/09/2023"}}]}
score_.score(None, req, dry_run=True)

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp9jj8204m
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp9jj8204m/_remote_module_non_scriptable.py


LLM model loading Open_Chat_3_5


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Fields Marked Asterisk (*) Can Be Validated On Proper Input 

Prompting has started
Total time taken = 371.7729585170746
prediction is 
:  
                    {
                        "MEASURE": {
                            "Quantity": [
                                {
                                    "ENTITY": "Quantity",
                                    "MEASURE CONSTRAINT": [
                                        {
                                            "COMPARISON VALUE": "",
                                            "COMPARSION OPERATOR": ""
                                        }
                                    ],
                                    "ADJECTIVE": [],
                                    "TONE": ""
                                }
                            ]
                        },
                        "DIMENSION": {
                            "Segment": [
                                {
                                    "ENTI

[<<ScoreResponse>>]

In [7]:
%%time
req = requests.Request()
req.json = {"payload":[{"question":"top 2 segments and bottom 3 sub-category basis quantity","context":context}]}
score_.score(None, req, dry_run=True)

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Fields Marked Asterisk (*) Can Be Validated On Proper Input 

Prompting has started
prediction is 
:  
                    {
                        "MEASURE": {
                            "Quantity": [
                                {
                                    "ENTITY": "Quantity",
                                    "MEASURE CONSTRAINT": [
                                        {
                                            "COMPARISON VALUE": "",
                                            "COMPARSION OPERATOR": ""
                                        }
                                    ],
                                    "ADJECTIVE": [],
                                    "TONE": ""
                                }
                            ]
                        },
                        "DIMENSION": {
                            "Segment": [
                                {
                                    "ENTITY": "Segment",
                     

[<<ScoreResponse>>]

In [75]:
%%time
req = requests.Request()
req.json = {"payload":[{"question":"top 2 segments basis quantity","context":context}]}
score_.score(None, req, dry_run=True)

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Fields Marked Asterisk (*) Can Be Validated On Proper Input 

promting has started
prediction is 
:  
                    {
                        "MEASURE": {
                            "segment": [
                                {
                                    "ENTITY": "segments",
                                    "MEASURE CONSTRAINT": [
                                        {
                                            "COMPARISON VALUE": "",
                                            "COMPARSION OPERATOR": ""
                                        }
                                    ],
                                    "ADJECTIVE": [],
                                    "TONE": ""
                                }
                            ]
                        },
                        "DIMENSION": {
                            "quantity": [
                                {
                                    "ENTITY": "quantity",
                     

[<<ScoreResponse>>]

In [10]:
model="/data/huggingface/cache/models--openchat--openchat_3.5/snapshots/2d39b2e68fc16446e1760700e0de14721434cbbf"
#model=score_.model_loaded

In [11]:
register_model(model,
               ScoreTemplateExample,
               "Open_Chat_iMQL_Generator_for_GPU_01",
               "IMQL_Generator_using_open_chat_3_5_for_GPU01",
               MLModelFlavours.pytorch,
               init_script="""sudo chmod -R 777 /home/mosaic-ai/.cache\\npip install torch \\npip install --upgrade transformers\\npip install accelerate"""
              )

INFO:root:PASS:: Mandatory Validation :: dict_keys(['name', 'description', 'flavour', 'scoring_func'])
INFO:root:PASS:: AlphaNumeric Validation :: dict_keys(['name'])
INFO:root:PASS:: Validation/IfPresentTypeCheck :: dict_keys(['schema', 'metadata_info', 'tags'])
INFO:root:PASS:: Validation/IfPresentSubfieldMustExist :: dict_keys(['kyd'])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "POST /registry/api/v1/ml-model/register HTTP/1.1" 200 2124
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/6ed23fdd-6fff-4354-9047-913b00cf916a HTTP/1.1" 200 2117
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/6ed23fdd-6fff-4354-9047-913b00cf916a HTTP/1.1"